# SECS Inversion and Visualization Test

This notebook demonstrates the Spherical Elementary Current Systems (SECS) inversion process and visualization techniques using magnetometer data. It processes 9 time steps from 2024-05-10 at 21:00 to 21:45 (in 5-minute intervals) and creates two types of visualizations:

1. **SECS Comparison Plots**: Shows the comparison between SECS-predicted magnetic fields and actual magnetometer observations
2. **RGB Visualization**: Displays the magnetic field components as RGB color channels

## Table of Contents

1. [Setup and Configuration](#1-setup-and-configuration)
   - Import libraries
   - Define parameters
   - Set up directories

2. [SECS Inversion Process](#2-secs-inversion-process)
   - Run SECS generation for selected time steps
   - Explanation of the SECS method

3. [SECS Comparison Visualization](#3-secs-comparison-visualization)
   - Generate comparison plots
   - Display 3×3 grid of comparison plots

4. [RGB Visualization](#4-rgb-visualization)
   - Generate RGB images from magnetic field components
   - Display 3×3 grid of RGB images

5. [Analysis and Interpretation](#5-analysis-and-interpretation)
   - Explanation of results
   - Discussion of patterns and features

## What are Spherical Elementary Current Systems (SECS)?

SECS is a mathematical method used to analyze and visualize ionospheric currents and their associated magnetic fields. The method works by:

1. **Setting up a grid of elementary current systems** in the ionosphere (about 110 km above Earth's surface)
2. **Measuring magnetic field variations** at ground-based magnetometer stations
3. **Solving an inverse problem** to find the current amplitudes that best explain the observed magnetic fields
4. **Calculating the resulting magnetic field** at any location using the estimated currents

This approach allows us to create a continuous map of magnetic field variations over a region, even in areas without direct measurements.

The inverse problem is mathematically challenging because it's underdetermined (more unknowns than measurements) and requires regularization techniques to obtain physically meaningful solutions.

## 1. Setup and Configuration

First, we'll import the necessary libraries and set up our configuration parameters.

In [1]:
# Import standard libraries
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from PIL import Image
import subprocess
from IPython.display import display, Image as IPImage

# Make sure the source directory is in the path so we can import our modules
sys.path.append('source')

# Import helper functions
from helper_functions import (
    create_dir, load_magnetic_component, create_rgb_image, save_rgb_image,
    filter_stations_by_grid, setup_secs_grid
)

# Set up matplotlib for better visualization
plt.rcParams['figure.figsize'] = (12, 10)
plt.rcParams['font.size'] = 12

### Define Parameters

Now we'll define the parameters for our analysis, including the time range, grid configuration, and data directories.

In [6]:
# Time range parameters
start_date_str = "2024-05-10 21:00"
end_date_str = "2024-05-10 21:08"
time_interval = 1  # minutes between time steps

# Parse dates
start_date = datetime.strptime(start_date_str, '%Y-%m-%d %H:%M')
end_date = datetime.strptime(end_date_str, '%Y-%m-%d %H:%M')

# Generate list of timestamps (9 time steps at 5-minute intervals)
timestamps = []
current = start_date
while current <= end_date:
    timestamps.append(current)
    current += timedelta(minutes=time_interval)

print(f"Analysis will process {len(timestamps)} timestamps:")
for ts in timestamps:
    print(f"  - {ts}")

# Grid parameters
grid_center = [17.0, 67.0]  # [longitude, latitude] in degrees (Northern Scandinavia)
grid_shape = [30, 18]       # [N-S points, E-W points]
grid_resolution = 100       # Grid resolution in kilometers

# Regularization parameters
l0 = 1e-2  # Zero-order (amplitude) regularization
l1 = 1e-2  # First-order (smoothness) regularization

# Mirror depth for induction nullification (9999 to disable)
mirror_depth = 1000  # in kilometers

# Visualization parameters
vmin_comp = -750  # Minimum value for comparison plots (nT)
vmax_comp = 750   # Maximum value for comparison plots (nT)
vmin_rgb = -1250  # Minimum value for RGB visualization (nT)
vmax_rgb = 1250   # Maximum value for RGB visualization (nT)

# Data directories
data_dir = "data"                # Base directory for magnetometer data
output_dir = "data/secs"         # Output directory for SECS data

# Create output directory if it doesn't exist
create_dir(output_dir)

Analysis will process 10 timestamps:
  - 2024-05-10 21:00:00
  - 2024-05-10 21:01:00
  - 2024-05-10 21:02:00
  - 2024-05-10 21:03:00
  - 2024-05-10 21:04:00
  - 2024-05-10 21:05:00
  - 2024-05-10 21:06:00
  - 2024-05-10 21:07:00
  - 2024-05-10 21:08:00
  - 2024-05-10 21:09:00


### Explanation of Parameters

- **Time Range**: We're analyzing data from 2024-05-10 at 21:00 to 21:45 in 5-minute intervals, giving us 9 time steps.

- **Grid Parameters**:
  - **Grid Center**: Located at 17°E, 67°N (Northern Scandinavia)
  - **Grid Shape**: 30×18 points (North-South × East-West)
  - **Grid Resolution**: 100 km between grid points

- **Regularization Parameters**:
  - **l0 (1e-2)**: Controls the amplitude of the solution (higher values produce smaller amplitudes)
  - **l1 (1e-2)**: Controls the smoothness of the solution (higher values produce smoother results)

- **Mirror Depth**: Set to 1000 km to account for induction effects in the Earth

- **Visualization Parameters**:
  - **Comparison Plots**: Values range from -750 to 750 nT
  - **RGB Visualization**: Values range from -1250 to 1250 nT

## 2. SECS Inversion Process

Now we'll run the SECS inversion process for our selected time steps. This involves:

1. Loading magnetometer data
2. Setting up a SECS grid
3. Calculating SECS basis matrices
4. Solving the inverse problem to find current amplitudes
5. Calculating the resulting magnetic field components

We'll use the `secs_gen.py` script to perform these steps.

In [5]:
def run_secs_generation():
    """
    Run the SECS generation process for the selected time range.
    
    This function calls the secs_gen.py script with our configured parameters.
    """
    # Construct command with all parameters
    cmd = [
        "python", "secs_gen.py",
        "--start_date", start_date_str,
        "--end_date", end_date_str,
        "--grid_center", str(grid_center[0]), str(grid_center[1]),
        "--grid_shape", str(grid_shape[0]), str(grid_shape[1]),
        "--grid_resolution", str(grid_resolution),
        "--l0", str(l0),
        "--l1", str(l1),
        "--mirror_depth", str(mirror_depth),
        "--time_resolution", str(time_interval),
        "--data_dir", data_dir,
        "--output_dir", output_dir
    ]
    
    print("Running SECS generation with command:")
    print(" ".join(cmd))
    
    # Run the command and capture output
    try:
        result = subprocess.run(cmd, check=True, capture_output=True, text=True)
        print("\nSECS generation completed successfully!")
        print("\nOutput:")
        print(result.stdout)
        return True
    except subprocess.CalledProcessError as e:
        print(f"\nError running SECS generation: {e}")
        print("\nError output:")
        print(e.stderr)
        return False

# Run SECS generation
secs_generation_success = run_secs_generation()

Running SECS generation with command:
python secs_gen.py --start_date 2024-05-10 21:00 --end_date 2024-05-10 21:45 --grid_center 17.0 67.0 --grid_shape 30 18 --grid_resolution 100 --l0 0.01 --l1 0.01 --mirror_depth 1000 --time_resolution 5 --data_dir data --output_dir data/secs

Error running SECS generation: Command '['python', 'secs_gen.py', '--start_date', '2024-05-10 21:00', '--end_date', '2024-05-10 21:45', '--grid_center', '17.0', '67.0', '--grid_shape', '30', '18', '--grid_resolution', '100', '--l0', '0.01', '--l1', '0.01', '--mirror_depth', '1000', '--time_resolution', '5', '--data_dir', 'data', '--output_dir', 'data/secs']' returned non-zero exit status 1.

Error output:
Traceback (most recent call last):
  File "/Users/akv020/Tensorflow/fennomag-net/source/secs_gen.py", line 262, in <module>
    main()
  File "/Users/akv020/Tensorflow/fennomag-net/source/secs_gen.py", line 140, in main
    raise ValueError("Could not load all magnetometer data files")
ValueError: Could not lo

### Understanding the SECS Inversion Process

The SECS inversion process we just ran involves several key steps:

1. **Data Loading**: The script loads X, Y, Z magnetic field components from magnetometer stations.

2. **Grid Setup**: A cubed-sphere grid is created in the ionosphere (110 km altitude) centered at our specified location.

3. **SECS Basis Calculation**: The script calculates how each elementary current system affects the magnetic field at each station location.

4. **Inverse Problem Solving**: The script solves the inverse problem to find current amplitudes that best explain the observed magnetic fields.

5. **Regularization**: Since there are more unknowns (current amplitudes) than measurements (station readings), regularization is applied:
   - **L0 regularization** (amplitude damping): Penalizes large current amplitudes
   - **L1 regularization** (smoothness): Penalizes rapid spatial variations in the current pattern

6. **Forward Calculation**: Once current amplitudes are determined, the magnetic field is calculated at all grid points.

7. **Output**: The script saves the magnetic field components (Be, Bn, Bu) for each timestamp.

The mathematical formulation of this process is:

1. **Forward problem**: B = G × I
   - B: Magnetic field measurements
   - G: SECS basis matrices
   - I: Current amplitudes

2. **Inverse problem with regularization**: I = (G^T × G + λ₀I + λ₁L^T × L)^(-1) × G^T × B
   - λ₀: Amplitude regularization parameter
   - λ₁: Smoothness regularization parameter
   - L: Differentiation operator
   - I: Identity matrix

## 3. SECS Comparison Visualization

Now we'll generate comparison plots that show:
- SECS-predicted horizontal magnetic field (black arrows)
- SECS-predicted vertical magnetic field (color map)
- Observed magnetometer horizontal field (blue arrows)
- Magnetometer station locations (blue dots)

We'll use the `secs_comp.py` script to create these visualizations.

In [ ]:
def run_secs_comparison():
    """
    Run the SECS comparison visualization for the selected time range.
    
    This function calls the secs_comp.py script with our configured parameters.
    """
    # Construct command with all parameters
    cmd = [
        "python", "source/secs_comp.py",
        "--start_date", start_date_str,
        "--end_date", end_date_str,
        "--data_dir", data_dir,
        "--secs_dir", output_dir,
        "--vmin", str(vmin_comp),
        "--vmax", str(vmax_comp)
    ]
    
    print("Running SECS comparison with command:")
    print(" ".join(cmd))
    
    # Run the command and capture output
    try:
        result = subprocess.run(cmd, check=True, capture_output=True, text=True)
        print("\nSECS comparison completed successfully!")
        print("\nOutput:")
        print(result.stdout)
        return True
    except subprocess.CalledProcessError as e:
        print(f"\nError running SECS comparison: {e}")
        print("\nError output:")
        print(e.stderr)
        return False

# Run SECS comparison
if secs_generation_success:
    secs_comparison_success = run_secs_comparison()
else:
    print("Skipping SECS comparison because SECS generation failed.")
    secs_comparison_success = False

### Display Comparison Plots

Now we'll display the comparison plots in a 3×3 grid. These plots show how well the SECS model predicts the observed magnetic field.

In [ ]:
def display_comparison_plots():
    """
    Display the SECS comparison plots in a 3×3 grid.
    """
    # Create figure with 3×3 subplots
    fig, axes = plt.subplots(3, 3, figsize=(18, 18))
    axes = axes.flatten()
    
    # Path to comparison plots
    comparison_dir = os.path.join(output_dir, str(start_date.year), 'comparison')
    
    # Load and display each plot
    for i, timestamp in enumerate(timestamps):
        if i >= len(axes):
            break
            
        # Construct filename
        filename = f"secs_comp_{timestamp.strftime('%Y%m%d_%H%M%S')}.png"
        file_path = os.path.join(comparison_dir, filename)
        
        if os.path.exists(file_path):
            # Load image
            img = plt.imread(file_path)
            
            # Display in subplot
            axes[i].imshow(img)
            axes[i].set_title(f"{timestamp.strftime('%Y-%m-%d %H:%M')}")
            axes[i].axis('off')
        else:
            axes[i].text(0.5, 0.5, f"Image not found:\n{filename}", 
                         ha='center', va='center', transform=axes[i].transAxes)
            axes[i].axis('off')
    
    # Hide any unused subplots
    for i in range(len(timestamps), len(axes)):
        axes[i].axis('off')
    
    plt.suptitle("SECS Comparison: Predicted vs. Observed Magnetic Fields", fontsize=20)
    plt.tight_layout(rect=[0, 0, 1, 0.97])
    plt.show()

# Display comparison plots
if secs_comparison_success:
    display_comparison_plots()
else:
    print("Cannot display comparison plots because SECS comparison failed.")

### Understanding the Comparison Plots

The comparison plots show several important features:

1. **Color Map (Background)**: Shows the vertical component of the magnetic field (Bu)
   - **Red**: Upward magnetic field (positive Bu)
   - **Blue**: Downward magnetic field (negative Bu)

2. **Black Arrows**: SECS-predicted horizontal magnetic field
   - Arrow direction shows the horizontal field direction
   - Arrow length shows the field strength

3. **Blue Dots**: Magnetometer station locations

4. **Blue Arrows**: Observed horizontal magnetic field at station locations
   - These should align with the black arrows if the SECS model is accurate

5. **Coastlines**: Shown in grey for geographic reference

The comparison between predicted (black) and observed (blue) arrows helps us evaluate how well the SECS model captures the actual magnetic field variations. Good agreement indicates that the model is successfully capturing the ionospheric current systems.

## 4. RGB Visualization

Now we'll generate RGB images from the SECS magnetic field components. Each component is mapped to a color channel:
- Red: Eastward component (Be)
- Green: Northward component (Bn)
- Blue: Upward component (Bu)

We'll use the `secs_viz.py` script to create these visualizations.

In [ ]:
def run_secs_visualization():
    """
    Run the SECS RGB visualization for the selected time range.
    
    This function calls the secs_viz.py script with our configured parameters.
    """
    # Construct command with all parameters
    cmd = [
        "python", "source/secs_viz.py",
        "--start_date", start_date_str,
        "--end_date", end_date_str,
        "--data_dir", output_dir,
        "--vmin", str(vmin_rgb),
        "--vmax", str(vmax_rgb)
    ]
    
    print("Running SECS visualization with command:")
    print(" ".join(cmd))
    
    # Run the command and capture output
    try:
        result = subprocess.run(cmd, check=True, capture_output=True, text=True)
        print("\nSECS visualization completed successfully!")
        print("\nOutput:")
        print(result.stdout)
        return True
    except subprocess.CalledProcessError as e:
        print(f"\nError running SECS visualization: {e}")
        print("\nError output:")
        print(e.stderr)
        return False

# Run SECS visualization
if secs_generation_success:
    secs_visualization_success = run_secs_visualization()
else:
    print("Skipping SECS visualization because SECS generation failed.")
    secs_visualization_success = False

### Display RGB Images

Now we'll display the RGB images in a 3×3 grid. These images provide a compact way to visualize all three magnetic field components simultaneously.

In [ ]:
def display_rgb_images():
    """
    Display the SECS RGB images in a 3×3 grid.
    """
    # Create figure with 3×3 subplots
    fig, axes = plt.subplots(3, 3, figsize=(18, 18))
    axes = axes.flatten()
    
    # Path to RGB images
    figures_dir = os.path.join(output_dir, str(start_date.year), 'figures')
    
    # Load and display each image
    for i, timestamp in enumerate(timestamps):
        if i >= len(axes):
            break
            
        # Construct filename
        filename = f"secs_rgb_{timestamp.strftime('%Y%m%d_%H%M%S')}.png"
        file_path = os.path.join(figures_dir, filename)
        
        if os.path.exists(file_path):
            # Load image
            img = plt.imread(file_path)
            
            # Display in subplot
            axes[i].imshow(img)
            axes[i].set_title(f"{timestamp.strftime('%Y-%m-%d %H:%M')}")
            axes[i].axis('off')
        else:
            axes[i].text(0.5, 0.5, f"Image not found:\n{filename}", 
                         ha='center', va='center', transform=axes[i].transAxes)
            axes[i].axis('off')
    
    # Hide any unused subplots
    for i in range(len(timestamps), len(axes)):
        axes[i].axis('off')
    
    plt.suptitle("SECS RGB Visualization: Magnetic Field Components as Colors", fontsize=20)
    plt.tight_layout(rect=[0, 0, 1, 0.97])
    plt.show()

# Display RGB images
if secs_visualization_success:
    display_rgb_images()
else:
    print("Cannot display RGB images because SECS visualization failed.")

### Understanding the RGB Visualization

The RGB visualization maps each magnetic field component to a color channel:

- **Red**: Eastward component (Be)
  - Bright red indicates strong eastward field
  - Dark/black indicates strong westward field

- **Green**: Northward component (Bn)
  - Bright green indicates strong northward field
  - Dark/black indicates strong southward field

- **Blue**: Upward component (Bu)
  - Bright blue indicates strong upward field
  - Dark/black indicates strong downward field

The resulting colors represent combinations of these components:

- **Yellow (Red + Green)**: Strong eastward and northward field (northeast direction)
- **Magenta (Red + Blue)**: Strong eastward and upward field
- **Cyan (Green + Blue)**: Strong northward and upward field
- **White (Red + Green + Blue)**: Strong field in all directions
- **Black**: Weak field in all directions

This visualization technique allows us to quickly identify patterns and features in the magnetic field that might not be obvious when looking at individual components.

## 5. Analysis and Interpretation

Now that we've generated and visualized the SECS results, let's analyze and interpret what we're seeing.

### Interpreting the Results

When analyzing the SECS results, look for these key features:

1. **Spatial Patterns**:
   - **Vortex structures**: Circular patterns in the horizontal field (arrows) often indicate field-aligned currents
   - **Parallel flows**: Aligned arrows indicate sheet currents
   - **Boundaries**: Sharp transitions in color or arrow direction may indicate boundaries between different current systems

2. **Temporal Evolution**:
   - How patterns change over the 45-minute period
   - Whether features move, intensify, or dissipate

3. **Model Accuracy**:
   - Compare blue arrows (observations) with black arrows (predictions)
   - Good alignment indicates the model is capturing the actual current systems
   - Discrepancies may indicate limitations in the model or data

4. **Physical Interpretation**:
   - **Eastward electrojet**: Strong east-west aligned currents (visible as north-south magnetic field)
   - **Westward electrojet**: Opposite of eastward electrojet
   - **Field-aligned currents**: Often visible as vortex structures
   - **Substorm features**: Rapid changes and intensifications during active periods

### Common Features in SECS Visualizations

Here are some common features you might observe in the visualizations:

1. **Auroral Electrojets**:
   - Strong east-west aligned currents in the auroral zone
   - Visible as predominantly red or cyan bands in the RGB visualization
   - In comparison plots, they appear as parallel arrows with strong vertical field (color) boundaries

2. **Field-Aligned Currents (FACs)**:
   - Connect the ionosphere to the magnetosphere
   - Appear as vortex structures in the horizontal field
   - Often occur in pairs (upward and downward currents)
   - In RGB visualization, they often appear as circular features with color transitions

3. **Substorm Features**:
   - Rapid intensifications of currents
   - Westward traveling surge (a characteristic feature during substorms)
   - Expansion and contraction of the auroral oval

4. **Quiet-Time Patterns**:
   - Weaker, more stable current systems
   - Less dramatic color variations in RGB visualization
   - Smoother, more organized arrow patterns in comparison plots

### Limitations and Considerations

When interpreting these results, keep in mind several limitations:

1. **Station Coverage**:
   - The accuracy of the SECS model depends on the number and distribution of magnetometer stations
   - Areas far from stations have less reliable results
   - The blue dots in comparison plots show where actual measurements exist

2. **Regularization Effects**:
   - The regularization parameters (l0 and l1) affect the solution
   - Too much regularization: Overly smooth, may miss real features
   - Too little regularization: Noisy, may show artifacts

3. **Ground Induction**:
   - The mirror method (mirror_depth parameter) attempts to account for currents induced in the Earth
   - This is an approximation and may not fully capture complex conductivity structures

4. **Ionospheric Height**:
   - The SECS model assumes currents flow at a fixed height (typically 110 km)
   - In reality, currents flow at various heights

5. **Time Resolution**:
   - Our 5-minute time steps may miss rapid variations
   - Some features may evolve faster than our sampling rate

## Conclusion

In this notebook, we've demonstrated the complete SECS workflow:

1. **SECS Inversion**: Calculating ionospheric currents from ground magnetometer data
2. **Comparison Visualization**: Evaluating the model against observations
3. **RGB Visualization**: Compactly representing all three magnetic field components

The SECS method provides a powerful tool for studying ionospheric currents and their associated magnetic fields. By solving the inverse problem with appropriate regularization, we can create continuous maps of magnetic field variations even in regions without direct measurements.

The two visualization approaches offer complementary insights:
- **Comparison plots** help evaluate model accuracy and show the vector nature of the field
- **RGB visualization** provides a compact representation that can reveal patterns not obvious in individual components

These techniques are valuable for studying space weather phenomena, including auroral electrojets, field-aligned currents, and geomagnetic substorms.

## Next Steps

If you want to explore further, here are some suggestions:

1. **Try different parameters**:
   - Adjust regularization parameters (l0, l1) to see how they affect the solution
   - Change the grid resolution or size
   - Modify the mirror depth to see how it affects the results

2. **Analyze different time periods**:
   - Compare quiet and disturbed geomagnetic conditions
   - Look at substorm onset and recovery phases
   - Examine seasonal or diurnal variations

3. **Advanced analysis**:
   - Calculate derived quantities like divergence-free and curl-free components
   - Estimate field-aligned current densities
   - Compare with satellite observations or auroral images

4. **Visualization enhancements**:
   - Create animations to better show temporal evolution
   - Add geographic features for better context
   - Implement interactive plots for exploration